# Questão 2 - Pedro Henrique Arruda Gonçalves

In [2]:
import pandas as pd

In [3]:
lst1 = ["Trabalhador 1", "Trabalhador 2", "Trabalhador 3", "Trabalhador 4", "Demanda"]
lst2 = [17, 14, 11, 19, 1]
lst3 = [22, 28, 21, 25, 1]
lst4 = [24, 18, 30, 21, 1]
lst5 = [15, 17, 20, 22, 1]
lst6 = [1, 1, 1, 1, 0]

df_aux = list(zip(lst1,lst2,lst3,lst4,lst5,lst6))

df = pd.DataFrame(df_aux)

df.columns = ["", "Fábrica 1", "Fábrica 2", "Fábrica 3", "Fábrica 4", "Capacidade"]

df # Iniciando o dataframe da questão com os respectivos tempos de trabalho
   # relacionado cada trabaçhador a uma determinada fábrica , e também com as capacidades dos trabalhadores e as demandas das fábricas 

,,Fábrica 1,Fábrica 2,Fábrica 3,Fábrica 4,Capacidade
0,Trabalhador 1,17,22,24,15,1
1,Trabalhador 2,14,28,18,17,1
2,Trabalhador 3,11,21,30,20,1
3,Trabalhador 4,19,25,21,22,1
4,Demanda,1,1,1,1,0


In [4]:
df = df.set_index(df.iloc[0:5,0]) # Alterar o índice
df.drop("", axis=1, inplace = True) # Excluir a primeira coluna
df

,Fábrica 1,Fábrica 2,Fábrica 3,Fábrica 4,Capacidade
,,,,,
Trabalhador 1,17,22,24,15,1
Trabalhador 2,14,28,18,17,1
Trabalhador 3,11,21,30,20,1
Trabalhador 4,19,25,21,22,1
Demanda,1,1,1,1,0


In [5]:
df.Capacidade.drop("Demanda")


Trabalhador 1    1
Trabalhador 2    1
Trabalhador 3    1
Trabalhador 4    1
Name: Capacidade, dtype: int64

In [6]:
df_capacidades = df.Capacidade.drop("Demanda")
df_capacidades


Trabalhador 1    1
Trabalhador 2    1
Trabalhador 3    1
Trabalhador 4    1
Name: Capacidade, dtype: int64

In [7]:
df_demandas = df.loc["Demanda"].drop("Capacidade")
df_demandas

Fábrica 1    1
Fábrica 2    1
Fábrica 3    1
Fábrica 4    1
Name: Demanda, dtype: int64

In [8]:
df_custos = df.iloc[:-1,:-1]
df_custos

,Fábrica 1,Fábrica 2,Fábrica 3,Fábrica 4
,,,,
Trabalhador 1,17,22,24,15
Trabalhador 2,14,28,18,17
Trabalhador 3,11,21,30,20
Trabalhador 4,19,25,21,22


## Pyomo

In [9]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [10]:
modelo = pyo.ConcreteModel()

### Criando índices

In [11]:
modelo.i = Set(initialize= df_custos.index, doc='Fábricas')
modelo.j = Set(initialize= df_custos.columns, doc = "CDs")

### Criando os parâmetros

In [12]:
modelo.capacidades = Param (modelo.i, initialize= df_capacidades, doc='Capacidades')
modelo.demandas = Param(modelo.j, initialize=df_demandas, doc='Demandas')

In [13]:
dicionario_custos = df_custos.stack().to_dict()
dicionario_custos

{('Trabalhador 1', 'Fábrica 1'): 17,
 ('Trabalhador 1', 'Fábrica 2'): 22,
 ('Trabalhador 1', 'Fábrica 3'): 24,
 ('Trabalhador 1', 'Fábrica 4'): 15,
 ('Trabalhador 2', 'Fábrica 1'): 14,
 ('Trabalhador 2', 'Fábrica 2'): 28,
 ('Trabalhador 2', 'Fábrica 3'): 18,
 ('Trabalhador 2', 'Fábrica 4'): 17,
 ('Trabalhador 3', 'Fábrica 1'): 11,
 ('Trabalhador 3', 'Fábrica 2'): 21,
 ('Trabalhador 3', 'Fábrica 3'): 30,
 ('Trabalhador 3', 'Fábrica 4'): 20,
 ('Trabalhador 4', 'Fábrica 1'): 19,
 ('Trabalhador 4', 'Fábrica 2'): 25,
 ('Trabalhador 4', 'Fábrica 3'): 21,
 ('Trabalhador 4', 'Fábrica 4'): 22}

In [14]:
modelo.custos = Param(modelo.i, modelo.j, initialize = dicionario_custos, doc='Custos')

### Criando as variáveis

In [15]:
modelo.x = Var(modelo.i, modelo.j, bounds=(0.0,None), doc='Quantidades Enviadas')

### Criando as restrições

In [16]:
def regra_capacidade(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.j) <= modelo.capacidades[i]

def regra_demanda(modelo, j):
    return sum(modelo.x[i,j] for i in modelo.i) >= modelo.demandas[j]


modelo.restr_capacidade = Constraint(modelo.i, rule = regra_capacidade)
modelo.restr_demanda = Constraint(modelo.j, rule = regra_demanda)

### Função Objetivo

In [17]:
def custo_total(modelo):
    return sum(modelo.custos[i,j] * modelo.x[i,j] for i in modelo.i for j in modelo.j)

modelo.Z = Objective(rule=custo_total, sense=minimize) 

In [18]:
opt = SolverFactory("gurobi")
opt.solve(modelo)

{'Problem': [{'Name': 'x17', 'Lower bound': 69.0, 'Upper bound': 69.0, 'Number of objectives': 1, 'Number of constraints': 9, 'Number of variables': 17, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 17, 'Number of nonzeros': 33, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.00104522705078125', 'Error rc': 0, 'Time': 0.24924349784851074}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [19]:
for v in modelo.component_data_objects(Var):
    print (str(v), v.value)
    
print("A  melhor alocação deo modo que as tarefas sejam realizadas foi de:", modelo.Z(), "minutos")

x[Trabalhador 1,Fábrica 1] 0.0
x[Trabalhador 1,Fábrica 2] 0.0
x[Trabalhador 1,Fábrica 3] 0.0
x[Trabalhador 1,Fábrica 4] 1.0
x[Trabalhador 2,Fábrica 1] 0.0
x[Trabalhador 2,Fábrica 2] 0.0
x[Trabalhador 2,Fábrica 3] 1.0
x[Trabalhador 2,Fábrica 4] 0.0
x[Trabalhador 3,Fábrica 1] 1.0
x[Trabalhador 3,Fábrica 2] 0.0
x[Trabalhador 3,Fábrica 3] 0.0
x[Trabalhador 3,Fábrica 4] 0.0
x[Trabalhador 4,Fábrica 1] 0.0
x[Trabalhador 4,Fábrica 2] 1.0
x[Trabalhador 4,Fábrica 3] 0.0
x[Trabalhador 4,Fábrica 4] 0.0
A  melhor alocação deo modo que as tarefas sejam realizadas foi de: 69.0  minutos
